In [1]:
import pandas as pd
import os
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact
import plotly.graph_objects as go
import ruptures as rpt
from itertools import combinations as comb

In [2]:
folder_path = 'data'
def calculate_mean_for_30s(df):
    df['Time (s)'] = pd.to_datetime(df['Time (s)'], unit='s')
    df.set_index('Time (s)', inplace=True)
    resampled_df = df.resample('30S').mean()
    return resampled_df

def calculate_var_for_30s(df):
    resampled_df = df.resample('30S').var()
    return resampled_df

def read_data(filename):
    accelerometer = pd.read_csv(os.path.join(folder_path, filename, 'Accelerometer.csv'),sep=';')
    gyroscope = pd.read_csv(os.path.join(folder_path, filename,'Gyroscope.csv'),sep=';')
    accelerometer.rename(columns={'Acceleration x (m/s^2)':'accelerometer_x','Acceleration y (m/s^2)':'accelerometer_y','Acceleration z (m/s^2)':'accelerometer_z'}, inplace=True)
    gyroscope.rename(columns={'Gyroscope x (rad/s)':'gyroscope_x','Gyroscope y (rad/s)':'gyroscope_y','Gyroscope z (rad/s)':'gyroscope_z'}, inplace=True)
    accelerometer =accelerometer[9000:-9000]
    gyroscope = gyroscope[9000:-9000]
    accelerometer_mean = calculate_mean_for_30s(accelerometer)
    gyroscope_mean = calculate_mean_for_30s(gyroscope)
    accelerometer_var = calculate_var_for_30s(accelerometer)
    gyroscope_var = calculate_var_for_30s(gyroscope)
    return accelerometer, gyroscope , accelerometer_mean, gyroscope_mean, accelerometer_var, gyroscope_var

def do_test(dataframe1, dataframe2):
    t_stat, p_val = stats.ttest_ind(dataframe1, dataframe2)
    return t_stat, p_val

In [3]:
accelerometer_lucas1, gyroscope_lucas1, accelerometer_lucas1_mean, gyroscope_lucas1_mean, accelerometer_lucas1_var, gyroscope_lucas1_var  = read_data('lucas1')
accelerometer_nick1, gyroscope_nick1, accelerometer_nick1_mean, gyroscope_nick1_mean, accelerometer_nick1_var, gyroscope_nick1_var  = read_data('nick1')
accelerometer_till1, gyroscope_till1, accelerometer_till1_mean, gyroscope_till1_mean, accelerometer_till1_var, gyroscope_till1_var  = read_data('till1')

In [4]:
t_stat, p_val = stats.ttest_ind(accelerometer_till1_mean['accelerometer_x'],accelerometer_lucas1_mean['accelerometer_x'])

In [5]:
dict_accel_var = {'accelerometer_lucas1_var': accelerometer_lucas1_var, 'accelerometer_nick1_var': accelerometer_nick1_var, 'accelerometer_till1_var': accelerometer_till1_var}
dict_accel_mean = {'accelerometer_lucas1_mean': accelerometer_lucas1_mean, 'accelerometer_nick1_mean': accelerometer_nick1_mean, 'accelerometer_till1_mean': accelerometer_till1_mean}
dict_gyro_var = {'gyroscope_lucas1_var': gyroscope_lucas1_var, 'gyroscope_nick1_var': gyroscope_nick1_var, 'gyroscope_till1_var': gyroscope_till1_var}
dict_gyro_mean = {'gyroscope_lucas1_mean': gyroscope_lucas1_mean, 'gyroscope_nick1_mean': gyroscope_nick1_mean, 'gyroscope_till1_mean': gyroscope_till1_mean}
dict_of_dfs = {'accel_mean': dict_accel_mean, 'gyro_mean':dict_gyro_mean,'accel_var':dict_accel_var, 'gyro_var' :dict_gyro_var}

In [6]:
list_columns_accelerometer = ['accelerometer_x', 'accelerometer_y', 'accelerometer_z']
list_columns_gyroscope = ['gyroscope_x', 'gyroscope_y', 'gyroscope_z']

# die art wie ich hier die results speichere suckt, sollte eigentlich nen table als index haben aber we
results = {}

search_string = 'accelerometer'

for df_name, df in dict_of_dfs.items():
    combinations = list(comb(df, 2))
    for combo in combinations:
        if search_string in combo[0]:
            for column in list_columns_accelerometer:
                t_stat, p_val = do_test(df[combo[0]][column], df[combo[1]][column])
                results[(combo[0],combo[1],column)] = [t_stat, p_val]
        else:
            for column in list_columns_gyroscope:
                t_stat, p_val = do_test(df[combo[0]][column], df[combo[1]][column])
                results[(combo[0],combo[1],column)] = [t_stat, p_val]

results_df = pd.DataFrame.from_dict(results, orient='index', columns=['T Statistic', 'P Value'])


In [9]:
pd.set_option('display.float_format', '{:.4f}'.format)
results_df

,T Statistic,P Value
"(accelerometer_lucas1_mean, accelerometer_nick1_mean, accelerometer_x)",47.3376,0.0000
"(accelerometer_lucas1_mean, accelerometer_nick1_mean, accelerometer_y)",-2.9718,0.0048
"(accelerometer_lucas1_mean, accelerometer_nick1_mean, accelerometer_z)",10.5200,0.0000
"(accelerometer_lucas1_mean, accelerometer_till1_mean, accelerometer_x)",45.9698,0.0000
"(accelerometer_lucas1_mean, accelerometer_till1_mean, accelerometer_y)",-8.3831,0.0000
"(accelerometer_lucas1_mean, accelerometer_till1_mean, accelerometer_z)",34.2570,0.0000
"(accelerometer_nick1_mean, accelerometer_till1_mean, accelerometer_x)",-21.7040,0.0000
"(accelerometer_nick1_mean, accelerometer_till1_mean, accelerometer_y)",0.5340,0.5956
"(accelerometer_nick1_mean, accelerometer_till1_mean, accelerometer_z)",3.2087,0.0023
"(gyroscope_lucas1_mean, gyroscope_nick1_mean, gyroscope_x)",5.2829,0.0000
